## Imports section

In [17]:
import pandas as pd
import pyodbc
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
from pycaret.classification import *
import requests

## Get data from SQL Server database

In [18]:
#SQL CONECTION
def conexao_sql_server():
    server = 'LAPTOP-TA7HV3O7\SQLEXPRESS'
    database = 'olist'
    username = 'XXXX'
    password = 'XXXX'
    string_conexao = 'Driver={SQL Server Native Client 11.0};Server='+server+';Database='+database+';UID='+username+';PWD='+password
    conexao = pyodbc.connect(string_conexao)
    return conexao


conn = conexao_sql_server()

In [19]:
#QUERY
query = '''
select 
    tb2.["seller_id"] as [id_vendedor],
    tb4.["seller_state"] as [estado],
    (datediff(day, min(tb1.["order_approved_at"]), '2018-09-03')+1) as [dias_na_base],
    (datediff(day, min(tb1.["order_approved_at"]), max(tb1.["order_approved_at"]))+1) as [dias_atividade],
    count(distinct(cast(tb1.["order_approved_at"] as date))) as [datas_distintas_de_venda],
    cast(count(distinct(cast(tb1.["order_approved_at"] as date))) as numeric) / (datediff(day, min(tb1.["order_approved_at"]), max(tb1.["order_approved_at"]))+1) as [%_de_atividade],
    cast(count(distinct(tb2.["order_id"])) as numeric) / count(distinct(cast(tb1.["order_approved_at"] as date))) as [vendas_por_dia_de_atividade],
    avg(cast(tb3.[review_score] as numeric)) as [avaliacao_media],
    count(distinct(tb2.["order_id"])) as [qtd_pedidos],
    count(tb2.["product_id"]) as [qtd_produtos],
    cast(count(tb2.["product_id"]) as numeric) / count(distinct(tb2.["order_id"])) as [media_produtos_por_pedido],
    sum(cast(tb2.["price"] as numeric)) as [receita_total],
    sum(cast(tb2.["price"] as numeric)) / count(distinct(tb2.["order_id"])) as  [media_valor_pedido_sem_frete],
    (sum(cast(tb2.["price"] as numeric)) + sum(cast(tb2.["freight_value"] as numeric))) / count(distinct(tb2.["order_id"])) as  [media_valor_pedido_com_frete],
    sum(cast(tb2.["freight_value"] as numeric)) / count(distinct(tb2.["order_id"])) as  [media_valor_frete],
    (convert(decimal(5,1),sum(case when tb1.["order_delivered_customer_date"] > tb1.["order_estimated_delivery_date"] then 1 else 0 end)))/count(tb2.["seller_id"]) as [prop_pedidos_atrasados],
    datediff(day, max(tb1.["order_approved_at"]), '2018-09-03') as [dias_sem_vender],
    avg(datediff(day, (tb1.["order_delivered_carrier_date"]), tb2.["shipping_limit_date"])) as [tempo_de_postagem]
    from olist_orders_dataset tb1

    left join olist_order_items_dataset tb2 
    on tb1.["order_id"] = tb2.["order_id"] 

    left join olist_order_reviews_dataset tb3
    on tb1.["order_id"] = tb3.[order_id]
    
    left join olist_sellers_dataset tb4
    on tb2.["seller_id"] = tb4.["seller_id"]
    
    where tb1.["order_status"] = 'delivered' and tb1.["order_approved_at"] NOT LIKE ''
    group by tb2.["seller_id"], tb4.["seller_state"]
    HAVING (datediff(day, min(tb1.["order_approved_at"]), '2018-09-03')+1) > 90
'''   

## Data processing

In [20]:
#query to df
vendedores = pd.read_sql(query, conn)
#rounding float64 columns
float_columns = list(vendedores.select_dtypes(include='float64').columns)
vendedores[float_columns] = np.round(vendedores[float_columns], decimals = 2)
#inputing the average value between 0 and 5 in empty evaluation fields
vendedores['avaliacao_media'] = vendedores['avaliacao_media'].fillna(2.5)
#column with the manual labels
vendedores['target'] = (vendedores['dias_sem_vender'] > 90)
#replacing values to 0 and 1
vendedores.loc[:, 'target'].replace({False: 0}, inplace=True)
vendedores.loc[:, 'target'].replace({True: 1}, inplace=True)

## Creating the model

In [21]:
#GRADIENT BOOSTING CLASSIFIER classification model setup
s = setup (vendedores[['dias_atividade', 'datas_distintas_de_venda',\
                       '%_de_atividade', 'vendas_por_dia_de_atividade',\
                       'qtd_pedidos', 'qtd_produtos',\
                       'media_produtos_por_pedido', 'receita_total',\
                       'media_valor_pedido_sem_frete', 'media_valor_pedido_com_frete',\
                       'media_valor_frete', 'prop_pedidos_atrasados', 'tempo_de_postagem', 'target']],
           target='target',
           numeric_features = ['dias_atividade', 'datas_distintas_de_venda',\
                       '%_de_atividade', 'vendas_por_dia_de_atividade',\
                       'qtd_pedidos', 'qtd_produtos',\
                       'media_produtos_por_pedido', 'receita_total',\
                       'media_valor_pedido_sem_frete', 'media_valor_pedido_com_frete',\
                       'media_valor_frete', 'prop_pedidos_atrasados', 'tempo_de_postagem'],
           normalize = True,
           normalize_method = 'minmax',
           data_split_stratify = True,
           fix_imbalance = True,
           silent=True,
           verbose=False
          )


#creating model
gbc = create_model('gbc', fold=5)
#finalizing modelo
final_model = finalize_model(gbc)
#saving
save_model(final_model, 'final_model')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7457,0.8351,0.6228,0.8062,0.7027,0.4868,0.4992
1,0.7746,0.8688,0.7006,0.8069,0.7500,0.5466,0.5511
2,0.7601,0.8504,0.7365,0.7593,0.7477,0.5192,0.5194
3,0.7681,0.8548,0.7289,0.7756,0.7516,0.5346,0.5355
4,0.7594,0.8559,0.7365,0.7593,0.7477,0.5179,0.5181
Mean,0.7616,0.8530,0.7051,0.7815,0.7399,0.5210,0.5247
SD,0.0097,0.0108,0.0432,0.0213,0.0187,0.0201,0.0175


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['dias_atividade',
                                                           'datas_distintas_de_venda',
                                                           '%_de_atividade',
                                                           'vendas_por_dia_de_atividade',
                                                           'qtd_pedidos',
                                                           'qtd_produtos',
                                                           'media_produtos_por_pedido',
                                                           'receita_total',
                                       

## Data preparation to API call

In [22]:
#1 sample to generate predictions
df = vendedores[['dias_atividade', 'datas_distintas_de_venda',\
                       '%_de_atividade', 'vendas_por_dia_de_atividade',\
                       'qtd_pedidos', 'qtd_produtos',\
                       'media_produtos_por_pedido', 'receita_total',\
                       'media_valor_pedido_sem_frete', 'media_valor_pedido_com_frete',\
                       'media_valor_frete', 'prop_pedidos_atrasados', 'tempo_de_postagem']].sample()

#turning it into json file
df_json = df.to_json(orient='records')
df_json

'[{"dias_atividade":81,"datas_distintas_de_venda":6,"%_de_atividade":0.07,"vendas_por_dia_de_atividade":1.17,"qtd_pedidos":7,"qtd_produtos":8,"media_produtos_por_pedido":1.14,"receita_total":1028.0,"media_valor_pedido_sem_frete":146.86,"media_valor_pedido_com_frete":168.14,"media_valor_frete":21.29,"prop_pedidos_atrasados":0.0,"tempo_de_postagem":1}]'

## Calling the API

In [23]:
url = 'http://192.168.0.228:5000/predict'
data = df_json
headers = {'Content-type': 'application/json'}

r = requests.post(url=url, data=data, headers=headers)

In [24]:
#returning status code from the api, if 200 its ok
print(r.status_code)
#printing the output of the API with the "prediction" column at the end
print(r.json())

200
[{'dias_atividade': 81, 'datas_distintas_de_venda': 6, '%_de_atividade': 0.07, 'vendas_por_dia_de_atividade': 1.17, 'qtd_pedidos': 7, 'qtd_produtos': 8, 'media_produtos_por_pedido': 1.14, 'receita_total': 1028.0, 'media_valor_pedido_sem_frete': 146.86, 'media_valor_pedido_com_frete': 168.14, 'media_valor_frete': 21.29, 'prop_pedidos_atrasados': 0.0, 'tempo_de_postagem': 1, 'prediction': 1}]


In [25]:
#transforming the json output into a dataframe
df = pd.DataFrame(r.json(), columns = r.json()[0].keys())
df

,dias_atividade,datas_distintas_de_venda,%_de_atividade,vendas_por_dia_de_atividade,qtd_pedidos,qtd_produtos,media_produtos_por_pedido,receita_total,media_valor_pedido_sem_frete,media_valor_pedido_com_frete,media_valor_frete,prop_pedidos_atrasados,tempo_de_postagem,prediction
0,81,6,0.07,1.17,7,8,1.14,1028.0,146.86,168.14,21.29,0.0,1,1
